# Milieugevolgen van de Uitstoot van Broeikasgassen
![image](images/home8.png)

In [ ]:
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
import numpy as np
from scipy.stats import pearsonr

nd_df = pd.read_csv('https://raw.githubusercontent.com/Elijah-1/IF-project/main/datasets/disaster-events%20new.csv')
ghg_df = pd.read_csv('https://raw.githubusercontent.com/Elijah-1/IF-project/main/datasets/transformed_data_separate_columns.csv')
oe_df = pd.read_csv('https://raw.githubusercontent.com/Elijah-1/IF-project/main/datasets/Oil_and_Earthquake.csv')

nd_df = nd_df[nd_df['Year'] >= 1970]
nd_df = nd_df[nd_df['Year'] != 2023]
nd_df = nd_df[nd_df['Year'] != 2024]

# emissions only all per year
ghg_all_df = ghg_df[ghg_df['Sector'] == 'GLOBAL TOTAL'][['Year', 'Emissions']]

# emissions excluding all per year
ghg_ex_df = ghg_df[ghg_df['Sector'] != 'GLOBAL TOTAL']

# natural disasters only all disasters per year
nd_all_df = nd_df[nd_df['Entity'] == 'All disasters'][['Year', 'Disasters']]

# natural disasters excluding all disasters {wildcard} per year
nd_ex_df = nd_df[~nd_df['Entity'].str.startswith('All disasters')]

nd_all_disasters_df = nd_df[nd_df['Entity'] == 'All disasters']

# merged
merged_all_df = pd.merge(ghg_all_df, nd_all_df, on='Year', how='inner')

disaster_counts = nd_ex_df.groupby('Entity')['Disasters'].sum().reset_index()

De wereld wordt steeds meer bedreigd door natuurrampen, een probleem dat verergerd wordt door de groeiende uitstoot van broeikasgassen. Mensen die in kwetsbare gebieden wonen, zijn ervan overtuigd dat de toename in frequentie en hevigheid van deze rampen direct verband houdt met de stijgende emissies. Dit heeft verwoestende effecten op hun levens, eigendommen en lokale economieën. Studies tonen aan dat broeikasgassen, veroorzaakt door menselijke activiteiten, de klimaatverandering versnellen. Dit leidt tot extremere weersomstandigheden zoals overstromingen en hittegolven. Anderen beweren echter dat veel van deze veranderingen te wijten zijn aan natuurlijke variaties in het klimaat en dat de impact van broeikasgassen wordt overdreven en dat adaptieve strategieën en technologieën voldoende zijn om eventuele negatieve effecten te mitigeren. Deze uiteenlopende perspectieven suggereren dat er verschillen in interpretatie en prioriteit zijn bij het aanpakken van de gevolgen van klimaatverandering.

Als belangrijkste databronnen gebruiken we de EDGAR-dataset, die ons gedetailleerde informatie verstrekt over de uitstoot van broeikasgassen van 1970-2022. Daarnaast maken we gebruik van een dataset die gegevens bevat over het aantal en de soorten natuurrampen die hebben plaatsgevonden van 1900-2024. Om de correlatie tussen natuurrampen en de uitstoot van broeikasgassen te begrijpen en te onderzoeken, zullen we factoren zoals emissies, natuurrampen en de Pearson-correlatiecoëfficiënt tussen de twee analyseren. Verder zullen we relevante literatuur raadplegen om de verschillende argumenten te onderbouwen.


### Trends in natuurrampen

Uit de stacked bar plot blijkt duidelijk dat de frequentie van natuurrampen de afgelopen jaren aanzienlijk is toegenomen (zie Figuur 1). Bij analyse van de grafiek valt op dat er tussen 1970 en 2000 een aanzienlijke stijging was in het aantal rampen per jaar, met opvallende pieken rond 2000 en 2010. Na 2000 stabiliseert het aantal rampen echter, zonder de sterke stijging die in de voorgaande jaren te zien was. Overstromingen en extreem weer vertoonden de grootste stijgingen, wat een aanzienlijk deel van de totale toename van rampen verklaart. Andere rampen, zoals aardbevingen en vulkanische activiteit, lieten een kleinere toename zien.

In [ ]:
disaster_types = nd_ex_df['Entity'].unique()
colors = px.colors.qualitative.T10[:len(disaster_types)]
color_map = {disaster: colors[i % len(colors)] for i, disaster in enumerate(disaster_types)}

fig_bar = go.Figure()

for disaster in disaster_types:
    disaster_data = nd_ex_df[nd_ex_df['Entity'] == disaster]
    fig_bar.add_trace(go.Bar(
        x=disaster_data['Year'],
        y=disaster_data['Disasters'],
        name=disaster,
        marker=dict(color=color_map[disaster])
    ))

fig_bar.update_layout(
    barmode='stack',
    title='Jaarlijkse natuurramp frequentie per type',
    xaxis=dict(title='Jaar', titlefont=dict(size=14), tickfont=dict(size=12)),
    yaxis=dict(title='Aantal rampen', titlefont=dict(size=14), tickfont=dict(size=12)),
    legend=dict(
        title='Ramptype',
        font=dict(size=12),
        orientation='h',
        yanchor='top',
        y=-0.15,  # Adjust this value to move the legend further below the graph
        xanchor='center',
        x=0.5
    ),
    height=600  # Increase height for better visibility
)

fig_bar.show()

> Figuur 1: Deze grafiek toont het aantal natuurrampen van het jaar 1970-2022, onderverdeeld naar type ramp met behulp van kleuren. Beweeg de muis erover om het precieze aantal rampen te zien. 

Wanneer we het aantal rampen per type weergeven in een donut chart (zie Figuur 2), blijkt dat extreem weer en overstromingen elk jaar een aanzienlijk deel van de natuurrampen vormen in vergelijking met andere typen rampen. Dit verschil wordt steeds groter door de jaren heen. Vooral overstromingen zijn sinds 2006 aanzienlijk toegenomen ten opzichte van de andere natuurrampen.

In [ ]:
years = nd_ex_df['Year'].unique()
years.sort()

disaster_counts_all_years = nd_ex_df.groupby('Entity')['Disasters'].sum().reset_index()

def get_disaster_counts(year):
    return nd_ex_df[nd_ex_df['Year'] == year].groupby('Entity')['Disasters'].sum().reset_index()

fig_pie = go.Figure()

fig_pie.add_trace(go.Pie(
    labels=disaster_counts_all_years['Entity'],
    values=disaster_counts_all_years['Disasters'],
    hole=0.8,
    marker=dict(colors=[color_map[disaster] for disaster in disaster_counts_all_years['Entity']]),
    textposition='outside',
    textinfo='percent'
))

frames = []
for year in years:
    counts = get_disaster_counts(year)
    frames.append(go.Frame(
        data=[go.Pie(
            labels=counts['Entity'],
            values=counts['Disasters'],
            hole=0.8,
            marker=dict(colors=[color_map[disaster] for disaster in counts['Entity']]),
            textposition='outside',
            textinfo='percent'
        )],
        name=str(year)
    ))

frames.append(go.Frame(
    data=[go.Pie(
        labels=disaster_counts_all_years['Entity'],
        values=disaster_counts_all_years['Disasters'],
        hole=0.8,
        marker=dict(colors=[color_map[disaster] for disaster in disaster_counts_all_years['Entity']]),
        textposition='outside',
        textinfo='percent'
    )],
    name='All years'
))

fig_pie.update_layout(
    title_text='Frequency of Different Disasters',
    updatemenus=[{
        'buttons': [{
            'args': [None, {'frame': {'duration': 500, 'redraw': True}, 'fromcurrent': True}],
            'label': 'Play',
            'method': 'animate'
        }, {
            'args': [[None], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate', 'transition': {'duration': 0}}],
            'label': 'Pause',
            'method': 'animate'
        }],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }],
    sliders=[{
        'active': len(years),
        'yanchor': 'top',
        'xanchor': 'left',
        'currentvalue': {
            'font': {'size': 20},
            'prefix': 'Year: ',
            'visible': True,
            'xanchor': 'right'
        },
        'transition': {'duration': 300, 'easing': 'cubic-in-out'},
        'pad': {'b': 10, 't': 50},
        'len': 0.9,
        'x': 0.1,
        'y': 0,
        'steps': [{
            'args': [[str(year)], {'frame': {'duration': 300, 'redraw': True}, 'mode': 'immediate', 'transition': {'duration': 300}}],
            'label': str(year),
            'method': 'animate'
        } for year in years] + [{
            'args': [['All years'], {'frame': {'duration': 300, 'redraw': True}, 'mode': 'immediate', 'transition': {'duration': 300}}],
            'label': 'All years',
            'method': 'animate'
        }]
    }],
    height=600
)

fig_pie.frames = frames

fig_pie.show()

> Figuur 2: De grafiek toont de verdeling van natuurrampen in een donut chart met percentages per ramp. Bij hover zie je de precieze aantallen. Met de slider bekijk je de jaarlijkse verdeling. Aan het einde van de slider zie je de cumulatieve verdeling van 1970 tot 2022.

Deze figuren laten zien dat weergerelateerde rampen, zoals overstromingen en extreem weer, de afgelopen decennia het sterkst zijn toegenomen. Dit sluit aan bij het algemene wetenschappelijke standpunt dat klimaatverandering, veroorzaakt door verhoogde broeikasgasemissies, de frequentie en ernst van weergerelateerde gebeurtenissen verergert (Ramón et al., 2015). Andere rampen, zoals aardbevingen en vulkanische activiteit, die minder vaak voorkomen (zie Figuur 2), vertoonden een kleinere toename. Rampen die direct gekoppeld zijn aan klimaatverandering, zoals overstromingen en extreem weer, zijn daarentegen aanzienlijk sterker gestegen.

Figuur 2 laat ook zien dat het aantal overstromingen tussen 2006 en 2022 een aanzienlijk groter percentage van het totale aantal rampen uitmaakt in vergelijking met de periode van 1970 tot 2000. Dit ondersteunt de theorie dat verhoogde broeikasgasemissies, die bijdragen aan klimaatverandering, verantwoordelijk zijn voor de toename van weergerelateerde rampen, met name overstromingen.

De gegevens uit figuur 1 laten ook zien dat het aantal natuurrampen tussen 2000 en 2022 niet verder is gestegen, ondanks de voortdurende toename van broeikasgasemissies (zie Figuur 3). Deze stabilisatie kan de indruk wekken dat er geen directe correlatie is tussen de stijgende emissies en het aantal rampen. Dit kan verschillende verklaringen hebben, zoals verbeterde aanpassings- en preventiemaatregelen, verbeterde infrastructuur en technologische vooruitgang die de impact van dergelijke rampen kunnen beperken of voorkomen. Een goed voorbeeld hiervan is Nederland, waar uitgebreide dijken en waterbeheersystemen effectief zijn gebleken in het voorkomen van overstromingen, ondanks de stijgende zeespiegel en hevige regenval als gevolg van klimaatverandering.


### Groei van emissies

Een belangrijke factor om te overwegen is de groei van de broeikasgasuitstoot. In 1970 bedroeg de totale broeikasgasuitstoot ongeveer 27 gigaton. Hoewel er af en toe dalingen waren, is de uitstoot over het algemeen aanzienlijk gestegen van 1970 tot 2022. In 2022 bedroeg de totale broeikasgasuitstoot ongeveer 53 gigaton, wat meer dan een verdubbeling is ten opzichte van 1970.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=ghg_all_df['Year'], y=ghg_all_df['Emissions'], mode='lines+markers', name='Emissions'))
fig.update_layout(
    title='Mondiale uitstoot van broeikasgassen door de jaren heen',
    xaxis_title='Jaar',
    yaxis_title='Emissies (Mton)',
    template='plotly_white',
)

fig.show()

> Figuur 3: Deze grafiek laat de trends zien in broeikasgasemissies van 1970 tot 2022. Over het algemeen vertoont de grafiek een toename van de broeikasgasuitstoot. Met name valt op dat de stijging tussen 2000 en 2022 aanzienlijk sterker is dan tussen 1970 en 2000.

Ondanks de enorme stijging in broeikasgasuitstoot, toont figuur 1 aan dat de groei van het aantal natuurrampen beperkt is gebleven. De grafiek laat zien dat er na 2000 weliswaar een minimale stijging is in het aantal natuurrampen, maar deze stijging is niet in verhouding met de toename van de broeikasgasuitstoot. Dit suggereert dat het aantal natuurrampen niet sterk is beïnvloed door de groei van de broeikasgasuitstoot.

### De mogelijke gevolgen van emissieproductie door de sectoren

In een tijdperk waarin klimaatverandering een van de grootste uitdagingen vormt, is het essentieel om te begrijpen hoe verschillende sectoren bijdragen aan de uitstoot van broeikasgassen en de daaruit voortvloeiende natuurrampen. De energietransitie, de afvalverwerking, en het transport zijn enkele van de sleutelsectoren die verantwoordelijk zijn voor significante emissieproductie.

In [ ]:
aggregated_df = ghg_ex_df.groupby(['Year', 'Sector'], as_index=False)['Emissions'].sum()

pivot_df = aggregated_df.pivot(index='Year', columns='Sector', values='Emissions')

fig = go.Figure()

for sector in pivot_df.columns:
    fig.add_trace(go.Scatter(
        x=pivot_df.index,
        y=pivot_df[sector],
        mode='lines+markers',
        name=sector,
    ))

fig.update_layout(
    title="Broeikasgasemissies per sector",
    xaxis_title="Jaar",
    yaxis_title="Emissies (Mton)",
    legend_title="Sector",
    hovermode="x",
    template="plotly_white",
    legend=dict(
        orientation="h",
        yanchor="top",
        y=-0.2,  # Adjust this value to move the legend further down
        xanchor="center",
        x=0.5
    )
)

fig.show()

> Figuur 4: De grafiek toont de uitstoot van broeikasgassen per sector van 1970 tot 2022, met sectoren zoals landbouw, gebouwen, brandstofwinning, industriële verbranding, de energiesector, processen, transport en afval.

Duidelijke trends zijn zichtbaar: de energiesector vertoont een significante toename in uitstoot, terwijl sectoren als afval en bouw relatief stabiel zijn gebleven.

Natuurrampen hebben diverse oorzaken. Zo kunnen bijvoorbeeld olieboringen aardbevingen veroorzaken. Om te begrijpen hoe emissies uit verschillende sectoren samenhangen met diverse typen rampen, onderzoeken we de Pearson-correlatie tussen emissies per sector en het voorkomen van deze rampen.

De Pearson-correlatie meet de sterkte en richting van de lineaire relatie tussen twee variabelen. Een waarde van 1 betekent een perfecte positieve correlatie, -1 een perfecte negatieve correlatie, en 0 betekent geen lineaire correlatie. Deze methode helpt ons te begrijpen of en hoe sterk emissies uit verschillende sectoren gerelateerd zijn aan het optreden van natuurrampen.


In [ ]:
sectors = ghg_ex_df['Sector'].unique()
disaster_types = nd_ex_df['Entity'].unique()

def calculate_pearson(df1, df2, sector):
    correlations = []
    for disaster in disaster_types:
        merged_df = pd.merge(
            df1[(df1['Sector'] == sector)][['Year', 'Emissions']],
            df2[(df2['Entity'] == disaster)][['Year', 'Disasters']],
            on='Year'
        )
        if not merged_df.empty:
            corr, _ = pearsonr(merged_df['Emissions'], merged_df['Disasters'])
            correlations.append(corr)
        else:
            correlations.append(None)
    return correlations

def calculate_all_disasters_pearson(df1, df2, sector):
    merged_df = pd.merge(
        df1[(df1['Sector'] == sector)][['Year', 'Emissions']],
        df2[['Year', 'Disasters']],
        on='Year'
    )
    if not merged_df.empty:
        corr, _ = pearsonr(merged_df['Emissions'], merged_df['Disasters'])
        return corr
    else:
        return None

initial_sector = sectors[0]
correlations = calculate_pearson(ghg_ex_df, nd_ex_df, initial_sector)
all_disasters_correlation = calculate_all_disasters_pearson(ghg_ex_df, nd_all_disasters_df, initial_sector)

colors = px.colors.qualitative.T10

fig = go.Figure(data=[
    go.Bar(name='Individual Disasters', x=disaster_types, y=correlations, marker_color=colors),
    go.Bar(name='All Disasters', x=['All Disasters'], y=[all_disasters_correlation], marker_color=colors[1])
])

fig.update_layout(
    title=f'Pearson correlatie over diverse sectoren',
    xaxis_title='Ramptype',
    yaxis_title='Pearson correlatiecoëfficiënt',
    barmode='group',
    showlegend=False  # Remove the legend
)

dropdown_buttons = [
    dict(label=sector,
         method='update',
         args=[{
             'y': [calculate_pearson(ghg_ex_df, nd_ex_df, sector),
                   [calculate_all_disasters_pearson(ghg_ex_df, nd_all_disasters_df, sector)]],
             'marker': [
                 {'color': colors[:len(disaster_types)]},  # Colors for individual disasters
                 {'color': [colors[1]]}  # Color for all disasters
             ],
             'title': f'Pearson Correlation for {sector} Sector'
         }])
    for sector in sectors
]

fig.update_layout(
    updatemenus=[
        dict(
            buttons=dropdown_buttons,
            direction='down',
            pad={'r': 10, 't': 10},
            showactive=True,
            x=0.1,
            xanchor='left',
            y=1.1,
            yanchor='top'
        )
    ]
)

fig.show()

> Figuur 5: De grafiek toont de Pearson-correlatiecoëfficiënten tussen verschillende typen natuurrampen en de emissies per sector. De correlaties geven aan hoe sterk de relatie is tussen de uitstoot in een specifieke sector en het optreden van een bepaald type ramp.  Met de knop kun je selecteren welke sector je wilt bekijken. Hogere waarden duiden op een sterkere positieve correlatie.

Extreem weer en overstromingen hebben consequent de hoogste positieve correlaties in meerdere sectoren, waaronder landbouw, brandstofwinning, industriële verbranding, de energiesector, processen, transport en afval. Dit wijst erop dat deze sectoren de grootste invloed hebben op deze rampen. In de landbouwsector vertonen extreem weer en overstromingen de grootste correlatie, terwijl droogte een gematigde positieve correlatie heeft. Over het geheel genomen benadrukken de grafieken dat extreem weer en overstromingen de grootste correlatie vormen.

Hoewel de Pearson correlatiecoëfficiënten een positief verband laten zien, kunnen we echter niet concluderen dat natuurrampen worden veroorzaakt door broeikasgassen, aangezien de correlatiecoëfficiënten vaak een erg lage waarde hebben. 

### Andere mogelijke oorzaken 

De oorzaken van natuurrampen zijn complex en kunnen niet eenvoudig worden vastgesteld. Naast de uitstoot van broeikasgassen per sector zijn er andere mogelijke factoren die natuurrampen kunnen veroorzaken, zoals aardbevingen door olieboringen en droogte door ontbossing. Door de onderstaande grafiek te bekijken, krijgen we een duidelijk overzicht van de veranderingen in het aantal aardbevingen en de olieproductie in de periode 1970-2014. 

In [ ]:
years = oe_df['year']
oil_production = oe_df['oil_prod']
number_of_disasters = oe_df['earthquake']

# Create the figure
fig = go.Figure()

# Add oil production line
fig.add_trace(go.Scatter(
    x=years,
    y=oil_production,
    mode='lines+markers',
    name='Olieproductie',
    yaxis='y1',
    line=dict(color='black'),
    marker=dict(color='black'),
    hovertemplate='Jaar: %{x}<br>Olieproductie: %{y}'
))

# Add number of disasters line
fig.add_trace(go.Scatter(
    x=years,
    y=number_of_disasters,
    mode='lines+markers',
    name='Aantal aardbevingen',
    yaxis='y2',
    line=dict(color='#008000'),  # Using hex code for green
    marker=dict(color='#008000'),  # Using hex code for green
    hovertemplate='Jaar: %{x}<br>Aantal aardbevingen: %{y}'
))

# Update layout for dual y-axis and legend at the bottom
fig.update_layout(
    title='Olieproductie en aantal aardbevingen door de jaren heen',
    xaxis=dict(title='Jaar'),
    yaxis=dict(
        title='Olieproductie (ton)',
        titlefont=dict(color='black'),
        tickfont=dict(color='black')
    ),
    yaxis2=dict(
        title='Aardbevingen',
        titlefont=dict(color='#008000'),  # Using hex code for green
        tickfont=dict(color='#008000'),  # Using hex code for green
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation='h',
        xanchor='center',
        yanchor='top'
    )
)

# Show the figure
fig.show()

> Figuur 6: De grafiek toont de olieproductie (zwarte lijn) en het aantal aardbevingen (groene lijn) van 1970 tot 2015, waarbij de linker y-as de olieproductie en de rechter y-as het aantal aardbevingen weergeeft. De lijnen en punten geven de jaarlijkse trends en waarden weer, met kleurcodering voor onderscheid. B staat voor miljard.

De grafiek laat zien dat zowel het aantal aardbevingen als de olieproductie op vergelijkbare manier schommelen. Een toename in olieproductie resulteert in een stijging van het aantal aardbevingen in de daaropvolgende jaren. Bovendien is aangetoond dat mijnbouwactiviteiten, waaronder olieboring en steenkoolproductie, op lange termijn invloed hebben op aardbevingen (Witkowski et al., 2024). Om verdere correlaties te onderzoeken en een betere vergelijking te maken met de invloed van broeikasgassen per sector op het aantal natuurrampen, berekenen we in dit geval ook de bijbehorende Pearson-correlatiecoëfficiënt. In dit geval is de coëfficiënt 0.400, wat een positief verband aangeeft en tevens hoger is dan de coëfficiënten van alle eerder besproken sectoren.

### Is er een relatie?

Vanaf het jaar 1970-2000 is het aantal natuurrampen extreem hard gestegen. Na het jaar 2000 is het aantal natuurrampen ongeveer gelijk gebleven, terwijl de broeikasgasuitstoot bleef toenemen. Deze grafiek laat het verband zien tussen natuurrampen en de broeikasgasuitstoot door de jaren heen.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=merged_all_df['Emissions'], 
    y=merged_all_df['Disasters'],
    mode='markers', 
    name='Verband tussen broeikasgasemissies en aantal natuurrampen',
    marker=dict(
        color=merged_all_df['Year'], 
        size=10, 
        colorscale='Viridis', 
        showscale=True
    )
))

fig.update_layout(
    title='Verband tussen broeikasgasemissies en aantal natuurrampen',
    xaxis_title='Emissies',
    yaxis_title='Rampen',
    template='plotly_white'
)

fig.show()

> Figuur 7: Deze grafiek toont de relatie tussen jaarlijkse broeikasgasemissies en het aantal natuurrampen van 1970 tot 2022.<br> Elk punt vertegenwoordigt een jaar, waarbij de kleurgradaties de tijdsprogressie aangeven.

Uit de grafiek blijkt dat zowel de broeikasgasuitstoot als het aantal natuurrampen toenam in de eerste paar jaren (1970-2000). Dit is te zien aan de kleurgradaties: de donkere kleuren (vroegere jaren) bevinden zich links met lagere waarden, terwijl de lichtere kleuren (latere jaren) rechts en hoger zijn, wat hogere waarden aangeeft. In de daaropvolgende jaren neemt de broeikasgasuitstoot nog steeds sterk toe, maar het aantal natuurrampen niet. Zoals te zien is in de grafiek blijven de punten ongeveer gelijk op de y-as, terwijl de kleur verandert van groen naar geel.

Vanuit de analyse van de gegevens over natuurrampen en broeikasgasemissies kunnen twee uiteenlopende conclusies worden getrokken. Ten eerste, wanneer we de periode van 1970 tot 2000 onder de loep nemen, blijkt dat de uitstoot van broeikasgassen een significante invloed heeft op de frequentie en intensiteit van natuurrampen. Gedurende deze dertig jaar is er een duidelijke correlatie waarneembaar tussen de stijging van broeikasgasemissies en de toename van natuurrampen zoals overstromingen en extreme temperaturen. De data suggereren dat de opwarming van de aarde, als gevolg van verhoogde broeikasgas concentraties, een directe rol speelt in het verergeren van deze natuurrampen.

In contrast hiermee, de periode van 2000 tot 2022 laat een ander beeld zien. In deze recente periode lijkt de uitstoot van broeikasgassen een minimale invloed te hebben op de frequentie en intensiteit van natuurrampen. Ondanks een voortdurende stijging van de broeikasgasemissies, is er geen duidelijke toename in het aantal natuurrampen.

### Samenvatting

De analyse toont aan dat natuurrampen, met name overstromingen en extreem weer, aanzienlijk zijn toegenomen van 1970 tot 2000, maar na 2000 gestabiliseerd zijn ondanks de stijgende uitstoot van broeikasgassen. Dit suggereert dat hoewel emissies bijdragen aan klimaatgerelateerde rampen, andere factoren zoals verbeterde aanpassingsmaatregelen hun impact kunnen verminderen. Emissies per sector correleren ook met verschillende soorten rampen, waarbij de energie- en landbouwsectoren de hoogste positieve correlaties vertonen. Over het algemeen waarnemen we echter lage positieve Pearson-correlaties, waardoor we geen definitieve conclusies kunnen trekken. Hoewel er na 2000 veel meer natuurrampen waren dan in de jaren ervoor, bleven de emissies redelijk geleidelijk stijgen. De complexe aard van rampen en andere invloedsfactoren, zoals olieproductie die tot aardbevingen leidt, geven aan dat emissies alleen de frequentie en intensiteit van natuurrampen niet volledig verklaren.

### Referenties

1. Ramón, L. V., Thomas, V., & Pablo, T. A. (2015, 1 december). Climate change and natural disasters. 
https://repositorio.uchile.cl/handle/2250/138715
2. Witkowski, W. T., Łucka, M., Guzy, A., Sudhaus, H., Barańska, A., & Hejmanowski, R. (2024). Impact of mining-induced seismicity on land subsidence occurrence. Remote Sensing Of Environment, 301, 113934. https://doi.org/10.1016/j.rse.2023.113934

### GitHub Repository

https://github.com/Elijah-1/IF-project